# Ansible is


 - configuration manager 
 - simple 
 - extensible via modules
 - written in python
 - broad community
 - many external tools
 - playbook repository
 - used by openstack, openshift & tonns of project
 
 
 # Configuration Manager
 
Explain infrastructure as code

- managing data centers through machine-readable definition files;
- install, configure, decommission physical and virtual resources;
- avoid manual processes;
- speed ++, costs --, risk --;
- declarative (describe the expected state) or imperative (describe the steps) approaches;
- push or pull model;

[![](https://mermaid.ink/img/pako:eNrNVMFu2zAM_RVBhyEB4sB2Yjfxhh7W9VB0Q4e1p0U9KDaTCLElT5LbuXH-fZQVrw2wATvuYpPPj-QjKetAc1UAzWgQBExaYUvIyI3caG6sbnLbaCDckCskMdlztprXO_L5G5NMFvA0GjH6Tq5N_d4_Nzzb8KAxoIk3S15bVb-lsD7wZNPx2CW-JJ2nOzlB3ZRloOFHA8Yi27nk5HZo1Go1emUjwUFGWKVbdEyuRW0NWhaquuQWnL1Txprxo68FciskuA68RbAz0l0i7e62I-Ks_V6sI3Qf8PvtXfcabJq1n4aHVjf8ilz7hF7eFrg2WJNLXrZGmH6E5ImXouBWKHnId5Dvjx4uoC5VW4G0TlZxVuBc0up8QVigEBpyl5E8fMR5aVWsGP2Kr6ZHUbgXhGt5-r2ZPzgDD-XxNTdDH39xB7YE-6z0PnhGFQX5R4zRR7chY1Hpg9_z_6nxtItSyP29bUsgM4KjV3vIHOMNHg_4VgNgBJ3QCnTFRYH_14FJQhi1O6iA0QzNAja8KS2jTB6R2tQoGa4Ld4xphruFCeWNVfetzAffcz4Jjqeioii3NIjWXH5XqhpI6NLsQH_SLJrPp_MwSaJwHs8Wy3A5oS3N4mUyTZNlOovTNI4X0cVxQl_6-HCaRkkcJlG6uIjnYRpGEwq9oC_-kujviuMvPMh_JQ?type=png)](https://mermaid.live/edit#pako:eNrNVMFu2zAM_RVBhyEB4sB2Yjfxhh7W9VB0Q4e1p0U9KDaTCLElT5LbuXH-fZQVrw2wATvuYpPPj-QjKetAc1UAzWgQBExaYUvIyI3caG6sbnLbaCDckCskMdlztprXO_L5G5NMFvA0GjH6Tq5N_d4_Nzzb8KAxoIk3S15bVb-lsD7wZNPx2CW-JJ2nOzlB3ZRloOFHA8Yi27nk5HZo1Go1emUjwUFGWKVbdEyuRW0NWhaquuQWnL1Txprxo68FciskuA68RbAz0l0i7e62I-Ks_V6sI3Qf8PvtXfcabJq1n4aHVjf8ilz7hF7eFrg2WJNLXrZGmH6E5ImXouBWKHnId5Dvjx4uoC5VW4G0TlZxVuBc0up8QVigEBpyl5E8fMR5aVWsGP2Kr6ZHUbgXhGt5-r2ZPzgDD-XxNTdDH39xB7YE-6z0PnhGFQX5R4zRR7chY1Hpg9_z_6nxtItSyP29bUsgM4KjV3vIHOMNHg_4VgNgBJ3QCnTFRYH_14FJQhi1O6iA0QzNAja8KS2jTB6R2tQoGa4Ld4xphruFCeWNVfetzAffcz4Jjqeioii3NIjWXH5XqhpI6NLsQH_SLJrPp_MwSaJwHs8Wy3A5oS3N4mUyTZNlOovTNI4X0cVxQl_6-HCaRkkcJlG6uIjnYRpGEwq9oC_-kujviuMvPMh_JQ)

Ansible uses the push model and supports both declarative and imperative approaches.

 # Advantages
 
 No agents: ansible copies python and all deployment scripts/modules to the target machine via ssh and executes them remotely. Some modules though require that target hosts contain specific python libraries.
 
 Jobs are executed in parallel, but you can configure for serialization using different strategies for speed up, rollout or other purposes: (link)
 
 ![Ansible architecture](https://cdn.pbrd.co/images/390yWc9H0.png "Ansible architecture")
 
 Authentication can be passwordless (ssh/pki, kerberos) or with password.
 
 Automation jobs (Playbooks) are described via YAML - a very concise and simple language. You can validate and lint files with yamllint and ansible-lint.
 
```
this_is:
  a: yaml

file:
- with dict
- a list
  
  
```
 
 Passwords are supported, but SSH keys with ssh-agent are one of the best ways to use Ansible. Though if you want to use Kerberos, that's good too. 
 
You have a lot of options! Root logins are not required, you can login as any user, and then su or sudo to any user.

In [1]:
cd /notebooks/exercise-00/

/notebooks/exercise-00


In [2]:
# Let's check our ansible directory
!tree

.
├── ansible.cfg
├── group_vars
│   ├── all
│   └── staging
├── inventory
├── mytemplate.j2
├── python-course-test.yml
├── site.yml
└── staging

1 directory, 8 files


## ansible.cfg 

It's the main configuration file. While all ansible are in `yaml`, ansible.cfg is in .ini format. Eg.

```
[stanza]
key = value

```

Let's check the content of a sample ansible.cfg:

  - there's a lot of stuff in there
  - there will be more ;)
  - for now **let's check only the uncommented ones**.

In [3]:
!cat ansible.cfg

#
# Always use ansible.cfg and inventory files to 
#  describe your deployment! 
#
# This will save time and helps your colleagues to
#  keep in touch with the environment.
#
[defaults]
# define our inventory file or ansible defaults
#  to /etc/ansible/hosts.
inventory = inventory

# It's just for testing purposes (or if you're on a secure cloud)
#  you may want to avoid typing yes to all host keys.
# Once you get host keys, you can comment this out and re-enable
#  checks.
host_key_checking = False

# When an ansible plabook fails, it creates a retry file which may
#  pollute the current directory ;) You can move this out using
#  further variables
retry_files_enabled = False

# Search password file in here
# vault_password_file = ~/.ssh/pin


# [ssh_connection]
# ssh_args = -F ./ssh_config/config -o ControlMaster=auto -o ControlPersist=30m
# control_path = ./ssh_config/ansible-%%r@%%h:%%p


## Inventories

a simple inventory file contains a static list of nodes to contact.

Generally, an [inventory can be static or dynamic](http://docs.ansible.com/ansible/intro_inventory.html), as we will see in the following lessons.



In [4]:
!cat inventory

#
# This inventory file contains a list of server to 
#  play with - divided in groups.
#
[course]
# this is the local machine where you run jupyter
# the tutorial just works with this one.
pythonforsysadmin_course_1  ansible_connection=local


# Another group of servers
# where we can pass optional arguments
# Homework: you can play with this group of host
#  once you exchange ssh-keys between the pythonforsysadmin_course_1
#  container and the pythonforsysadmin_ansible_* ones._
[ansible]
172.17.0.[5:7]   

#
# Besides, ansible has two predefined groups: 
#   - all 
#   - ungrouped


In [ ]:
# You can have many inventory files
!cat staging

## Environment variables

**N.B. ansible environment variables are not related with process environment**

You defined your host groups in the environment, eg:

  - course
  - ansible
  - staging
  
Ansible defines [two default groups: all and ungrouped](http://docs.ansible.com/ansible/intro_inventory.html#default-groups).

You can assign variables to all hosts using the `all` group.

In [5]:
# group_vars - a directory containing environment files for various host groups.
!tree group_vars

group_vars
├── all
└── staging

0 directories, 2 files


In [7]:
# I set  env_name in two different files
!grep env_name -r group_vars/
!cat group_vars/staging

group_vars/all:env_name: default
group_vars/staging:env_name: staging
#
# Pur here the variables for the staging inventory.
#
env_name: staging


In [ ]:
# The debug module (-m debug) shows variables' content or dumps messages.
#   by default uses the inventory set into ansible.cfg, thus writing
!ansible all -m debug -a 'var=env_name'


#### Exercise

Dump env_name tied to the staging inventory.

  - which is the expected output?
  - what ties the "staging" inventory file to group_vars/staging?


In [ ]:
# Solution

!ansible all -i staging -m debug -a 'var=env_name'

In [ ]:
# Use this cell for the exercise

#### Exercise

In [ ]:
#
# Read the inventory and try to predict the output of
#
!ansible course -i staging -m debug -a 'var=proxy_env'